In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dense, ELU, Flatten, Dropout, BatchNormalization,Add,GlobalAveragePooling2D,Softmax, Concatenate,add
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint,EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import math
from sklearn.model_selection import train_test_split
import random
from PIL import Image

In [2]:
train_datas = pd.read_csv("train.csv")
val_datas = pd.read_csv("Dig-MNIST.csv")

In [3]:
print(train_datas.shape)
print(val_datas.shape)
#train_datas = train_datas / 255.0
#val_datas = val_datas / 255.0

#concat train and val
datas = pd.concat([train_datas,val_datas],axis=0)
print(datas.shape)
datas_X = np.array(datas.drop("label",axis=1),dtype=np.float32)
datas_Y = np.array(datas[["label"]],dtype=np.int32)
train_X,val_X,train_Y,val_Y = train_test_split(datas_X,datas_Y,test_size=0.2,shuffle=True)
train_X2,val_X2,train_Y2,val_Y2 = train_test_split(datas_X,datas_Y,test_size=0.2,shuffle=True)
train_X3,val_X3,train_Y3,val_Y3 = train_test_split(datas_X,datas_Y,test_size=0.2,shuffle=True)

(60000, 785)
(10240, 785)
(70240, 785)


In [4]:
train_X = np.reshape(train_X,(-1,28,28,1))
val_X = np.reshape(val_X,(-1,28,28,1))
train_X2 = np.reshape(train_X2,(-1,28,28,1))
val_X2 = np.reshape(val_X2,(-1,28,28,1))
train_X3 = np.reshape(train_X3,(-1,28,28,1))
val_X3 = np.reshape(val_X3,(-1,28,28,1))

print(train_X.shape,train_X.dtype)
print(train_Y.shape,train_Y.dtype)
print(val_X.shape,val_X.dtype)
print(val_Y.shape,val_Y.dtype)

(56192, 28, 28, 1) float32
(56192, 1) int32
(14048, 28, 28, 1) float32
(14048, 1) int32


In [5]:
row,col,ch= (28,28,1)


mean = 0
var = 10
sigma = var**0.4
gauss = np.random.normal(mean,sigma,(train_X.shape[0],row,col,ch))
gauss = gauss.reshape(train_X.shape[0],row,col,ch)
noisy2 = train_X + gauss
noisy2[noisy2>255] = 255
noisy2[noisy2<0] = 0

tmp_y = train_Y

train_X = np.concatenate((train_X,noisy2))
train_Y = np.concatenate((train_Y,tmp_y))

In [6]:
row,col,ch= (28,28,1)

mean = 0
var = 10
sigma = var**0.4
gauss = np.random.normal(mean,sigma,(train_X2.shape[0],row,col,ch))
gauss = gauss.reshape(train_X2.shape[0],row,col,ch)
noisy2 = train_X2 + gauss
noisy2[noisy2>255] = 255
noisy2[noisy2<0] = 0

tmp_y = train_Y2

train_X2 = np.concatenate((train_X2,noisy2))
train_Y2 = np.concatenate((train_Y2,tmp_y))

In [7]:
row,col,ch= (28,28,1)

mean = 0
var = 10
sigma = var**0.4
gauss = np.random.normal(mean,sigma,(train_X3.shape[0],row,col,ch))
gauss = gauss.reshape(train_X3.shape[0],row,col,ch)
noisy2 = train_X3 + gauss
noisy2[noisy2>255] = 255
noisy2[noisy2<0] = 0

tmp_y = train_Y3

train_X3 = np.concatenate((train_X3,noisy2))
train_Y3 = np.concatenate((train_Y3,tmp_y))

In [41]:
input_layer = Input(shape=(28,28,1))
net = Conv2D(64, (3,3), padding='same')(input_layer)
net = BatchNormalization(momentum=0.5, epsilon=1e-5, gamma_initializer="uniform")(net)
net = ELU(alpha=0.1)(net)

net = Conv2D(64,  (3,3), padding='same')(net)
net = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(net)
net = ELU(alpha=0.1)(net)

net = MaxPooling2D(2, 2)(net)
net = Dropout(0.2)(net)

net = Conv2D(128,  (3,3), padding='same')(net)
net = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(net)
net = ELU(alpha=0.1)(net)


net = Conv2D(128,  (3,3), padding='same')(net)
net = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net)
net = ELU(alpha=0.1)(net)


net = MaxPooling2D(2, 2)(net)
net = Dropout(0.2)(net)

# net = Conv2D(196,  (3,3), padding='same')(net)
# net = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(net)
# net = ELU(alpha=0.1)(net)


net = Conv2D(196,  (1,1), padding='same')(net)
net = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net)
net = ELU(alpha=0.1)(net)


net = Conv2D(10,  (1,1), padding='same')(net)
net = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net)
net = ELU(alpha=0.1)(net)

net = GlobalAveragePooling2D()(net)

output = Dense(10, activation='softmax')(net)
model = Model(inputs=input_layer,outputs=output)
print(model.summary())

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_25 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
elu_25 (ELU)                 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
batch_normalization_26 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
elu_26 (ELU)                 (None, 28, 28, 64)        0   

In [42]:
input_layer = Input(shape=(28,28,1))
net2 = Conv2D(64, (3,3), padding='same')(input_layer)
net2 = BatchNormalization(momentum=0.5, epsilon=1e-5, gamma_initializer="uniform")(net2)
net2 = ELU(alpha=0.1)(net2)



net2 = Conv2D(64,  (3,3), padding='same')(net2)
net2 = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(net2)
net2 = ELU(alpha=0.1)(net2)


net2 = MaxPooling2D(2, 2)(net2)
net2 = Dropout(0.2)(net2)

net2 = Conv2D(128,  (3,3), padding='same')(net2)
net2 = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(net2)
net2 = ELU(alpha=0.1)(net2)


net2 = Conv2D(128,  (3,3), padding='same')(net2)
net2 = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net2)
net2 = ELU(alpha=0.1)(net2)


net2 = MaxPooling2D(2, 2)(net2)
net2 = Dropout(0.2)(net2)

net2 = Conv2D(256,  (5,5), padding='same')(net2)
net2 = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(net2)
net2 = ELU(alpha=0.1)(net2)


net2 = Conv2D(256,  (1,1), padding='same')(net2)
net2 = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net2)
net2 = ELU(alpha=0.1)(net2)


net2 = Conv2D(10,  (1,1), padding='same')(net2)
net2 = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net2)
net2 = ELU(alpha=0.1)(net2)

net2 = GlobalAveragePooling2D()(net2)

output = Dense(10, activation='softmax')(net2)
model2 = Model(inputs=input_layer,outputs=output)
print(model2.summary())

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_31 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
elu_31 (ELU)                 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
batch_normalization_32 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
elu_32 (ELU)                 (None, 28, 28, 64)        0   

In [43]:
input_layer = Input(shape=(28,28,1))
net3 = Conv2D(64, (3,3), padding='same')(input_layer)
net3 = BatchNormalization(momentum=0.5, epsilon=1e-5, gamma_initializer="uniform")(net3)
net3 = ELU(alpha=0.1)(net3)



net3 = Conv2D(64,  (3,3), padding='same')(net3)
net3 = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(net3)
net3 = ELU(alpha=0.1)(net3)


net3 = MaxPooling2D(2, 2)(net3)
net3 = Dropout(0.2)(net3)

net3 = Conv2D(128,  (3,3), padding='same')(net3)
net3 = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(net3)
net3 = ELU(alpha=0.1)(net3)


net3 = Conv2D(128,  (3,3), padding='same')(net3)
net3 = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net3)
net3 = ELU(alpha=0.1)(net3)


net3 = MaxPooling2D(2, 2)(net3)
net3 = Dropout(0.2)(net3)

net3 = Conv2D(196,  (1,1), padding='same')(net3)
net3 = BatchNormalization(momentum=0.2, epsilon=1e-5, gamma_initializer="uniform")(net3)
net3 = ELU(alpha=0.1)(net3)


net3 = Conv2D(196,  (1,1), padding='same')(net3)
net3 = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net3)
net3 = ELU(alpha=0.1)(net3)


net3 = Conv2D(10,  (1,1), padding='same')(net3)
net3 = BatchNormalization(momentum=0.1, epsilon=1e-5, gamma_initializer="uniform")(net3)
net3 = ELU(alpha=0.1)(net3)

net3 = GlobalAveragePooling2D()(net3)

output = Dense(10, activation='softmax')(net3)
model3 = Model(inputs=input_layer,outputs=output)
print(model3.summary())

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
batch_normalization_38 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
elu_38 (ELU)                 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 28, 28, 64)        36928     
_________________________________________________________________
batch_normalization_39 (Batc (None, 28, 28, 64)        256       
_________________________________________________________________
elu_39 (ELU)                 (None, 28, 28, 64)        0   

In [44]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model3.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy",metrics=["accuracy"])

In [12]:
datagen = ImageDataGenerator(
    rotation_range = 5,
    width_shift_range = 0.12,
    height_shift_range = 0.12,
    shear_range = 0.1,
    zoom_range = 0.15,
    horizontal_flip = False,
    rescale=1/255.0)

In [45]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


earlyStopping = EarlyStopping(monitor='val_acc', patience=10)
learning_rate_reduction = ReduceLROnPlateau(monitor='loss', 
                                            patience=3, 
                                            factor=0.4, 
                                            min_lr=0.000001)

epochs = 60
batch_size = 1024

In [46]:
history = model.fit_generator(datagen.flow(train_X,train_Y, batch_size=batch_size),
                                    epochs = epochs, 
                                    steps_per_epoch = 100,
                                    validation_data = (val_X/255.0,val_Y),
                                    callbacks=[learning_rate_reduction],
                                    )

Epoch 1/60
100/100 [==============================] - 19s 193ms/step - loss: 2.1985 - acc: 0.4911 - val_loss: 2.0058 - val_acc: 0.7077
Epoch 2/60
100/100 [==============================] - 16s 162ms/step - loss: 1.8058 - acc: 0.7550 - val_loss: 1.4809 - val_acc: 0.8632
Epoch 3/60
100/100 [==============================] - 16s 163ms/step - loss: 1.3027 - acc: 0.9098 - val_loss: 0.9485 - val_acc: 0.9681
Epoch 4/60
100/100 [==============================] - 16s 163ms/step - loss: 0.8624 - acc: 0.9647 - val_loss: 0.6047 - val_acc: 0.9742
Epoch 5/60
100/100 [==============================] - 16s 163ms/step - loss: 0.5636 - acc: 0.9751 - val_loss: 0.3757 - val_acc: 0.9823
Epoch 6/60
100/100 [==============================] - 16s 164ms/step - loss: 0.3724 - acc: 0.9805 - val_loss: 0.2419 - val_acc: 0.9853
Epoch 7/60
100/100 [==============================] - 16s 164ms/step - loss: 0.2612 - acc: 0.9833 - val_loss: 0.1657 - val_acc: 0.9870
Epoch 8/60
100/100 [==============================] - 1

In [47]:
history2 = model2.fit_generator(datagen.flow(train_X2,train_Y2, batch_size=batch_size),
                                    epochs = epochs, 
                                    steps_per_epoch = 100,
                                    validation_data = (val_X2/255.0,val_Y2),
                                    callbacks=[learning_rate_reduction],
                                    )

Epoch 1/60
100/100 [==============================] - 25s 253ms/step - loss: 2.1097 - acc: 0.7359 - val_loss: 1.8235 - val_acc: 0.9672
Epoch 2/60
100/100 [==============================] - 20s 203ms/step - loss: 1.5542 - acc: 0.9676 - val_loss: 1.1735 - val_acc: 0.9815
Epoch 3/60
100/100 [==============================] - 20s 199ms/step - loss: 0.9406 - acc: 0.9851 - val_loss: 0.6268 - val_acc: 0.9895
Epoch 4/60
100/100 [==============================] - 20s 199ms/step - loss: 0.5109 - acc: 0.9892 - val_loss: 0.3432 - val_acc: 0.9905
Epoch 5/60
100/100 [==============================] - 20s 200ms/step - loss: 0.2875 - acc: 0.9913 - val_loss: 0.1998 - val_acc: 0.9925
Epoch 6/60
100/100 [==============================] - 20s 200ms/step - loss: 0.1771 - acc: 0.9922 - val_loss: 0.1265 - val_acc: 0.9926
Epoch 7/60
100/100 [==============================] - 20s 200ms/step - loss: 0.1206 - acc: 0.9931 - val_loss: 0.0930 - val_acc: 0.9925
Epoch 8/60
100/100 [==============================] - 2

In [49]:
history3 = model3.fit_generator(datagen.flow(train_X2,train_Y2, batch_size=batch_size),
                                    epochs = epochs, 
                                    steps_per_epoch = 100,
                                    validation_data = (val_X2/255.0,val_Y2),
                                    callbacks=[learning_rate_reduction],
                                    )

Epoch 1/60
100/100 [==============================] - 17s 172ms/step - loss: 0.0048 - acc: 0.9990 - val_loss: 0.0183 - val_acc: 0.9952
Epoch 2/60
100/100 [==============================] - 17s 172ms/step - loss: 0.0044 - acc: 0.9990 - val_loss: 0.0188 - val_acc: 0.9956
Epoch 3/60
100/100 [==============================] - 17s 172ms/step - loss: 0.0045 - acc: 0.9990 - val_loss: 0.0177 - val_acc: 0.9957
Epoch 4/60
100/100 [==============================] - 17s 174ms/step - loss: 0.0040 - acc: 0.9992 - val_loss: 0.0176 - val_acc: 0.9960
Epoch 5/60
100/100 [==============================] - 17s 173ms/step - loss: 0.0043 - acc: 0.9990 - val_loss: 0.0191 - val_acc: 0.9954
Epoch 6/60
100/100 [==============================] - 17s 173ms/step - loss: 0.0042 - acc: 0.9991 - val_loss: 0.0205 - val_acc: 0.9953
Epoch 7/60
100/100 [==============================] - 17s 175ms/step - loss: 0.0043 - acc: 0.9989 - val_loss: 0.0183 - val_acc: 0.9952
Epoch 8/60
100/100 [==============================] - 1

In [34]:
train_DNN = np.zeros( (train_X.shape[0],30))
train_model1 = model.predict(train_X/255.)
train_model2 = model2.predict(train_X/255.)
train_model3 = model3.predict(train_X/255.)

train_model1 = train_model1.reshape(train_model1.shape[0],10)
train_model2 = train_model2.reshape(train_model2.shape[0],10)
train_model3 = train_model3.reshape(train_model3.shape[0],10)


train_DNN = np.concatenate((train_model1, train_model2), axis=1)
train_DNN = np.concatenate((train_DNN, train_model3), axis=1)


In [35]:
val_DNN = np.zeros( (val_X.shape[0],30))
results1 = np.zeros( (val_X.shape[0],10))
results2 = np.zeros( (val_X.shape[0],10))
results3 = np.zeros( (val_X.shape[0],10))

val_model1 =   model.predict(val_X/255.)
val_model2 =  model2.predict(val_X/255.)
val_model3 =  model3.predict(val_X/255.)


val_model1 = val_model1.reshape(val_model1.shape[0],10)
val_model2 = val_model2.reshape(val_model2.shape[0],10)
val_model3 = val_model3.reshape(val_model3.shape[0],10)

val_DNN = np.concatenate((val_model1, val_model2), axis=1)
val_DNN = np.concatenate((val_DNN, val_model3), axis=1)


In [36]:
print(train_DNN.shape)
print(val_DNN.shape)

(112384, 30)
(14048, 30)


In [37]:
input_layer = Input(shape=(30,))

net_dnn = Dense(18)(input_layer)
net_dnn = BatchNormalization()(net_dnn)
net_dnn = ELU(alpha=0.1)(net_dnn)

net = Dropout(0.2)(net)
net_dnn = Dense(18)(net_dnn)
net_dnn = BatchNormalization()(net_dnn)
net_dnn = ELU(alpha=0.1)(net_dnn)

output = Dense(10, activation='softmax')(net_dnn)
model_dnn = Model(inputs=input_layer,outputs=output)
print(model_dnn.summary())

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 30)]              0         
_________________________________________________________________
dense_6 (Dense)              (None, 18)                558       
_________________________________________________________________
batch_normalization_23 (Batc (None, 18)                72        
_________________________________________________________________
elu_23 (ELU)                 (None, 18)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 18)                342       
_________________________________________________________________
batch_normalization_24 (Batc (None, 18)                72        
_________________________________________________________________
elu_24 (ELU)                 (None, 18)                0   

In [38]:
model_dnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="sparse_categorical_crossentropy",metrics=["accuracy"])
history_dnn = model_dnn.fit(train_DNN,train_Y, batch_size=batch_size,
                                    epochs = 24, 
                                    #steps_per_epoch = 72,
                                    validation_data = (val_DNN,val_Y),
                                    #callbacks=[learning_rate_reduction],
                                    )

Train on 112384 samples, validate on 14048 samples
Epoch 1/24
112384/112384 [==============================] - 1s 13us/sample - loss: 1.3040 - acc: 0.6892 - val_loss: 1.4931 - val_acc: 0.9981
Epoch 2/24
112384/112384 [==============================] - 1s 8us/sample - loss: 0.2924 - acc: 0.9995 - val_loss: 0.5747 - val_acc: 0.9979
Epoch 3/24
112384/112384 [==============================] - 1s 8us/sample - loss: 0.1016 - acc: 0.9995 - val_loss: 0.1617 - val_acc: 0.9981
Epoch 4/24
112384/112384 [==============================] - 1s 9us/sample - loss: 0.0518 - acc: 0.9995 - val_loss: 0.0600 - val_acc: 0.9981
Epoch 5/24
112384/112384 [==============================] - 1s 8us/sample - loss: 0.0317 - acc: 0.9996 - val_loss: 0.0334 - val_acc: 0.9981
Epoch 6/24
112384/112384 [==============================] - 1s 8us/sample - loss: 0.0215 - acc: 0.9996 - val_loss: 0.0239 - val_acc: 0.9981
Epoch 7/24
112384/112384 [==============================] - 1s 8us/sample - loss: 0.0154 - acc: 0.9997 - val

In [39]:
val_DNN = np.zeros( (val_X.shape[0],30))
results1 = np.zeros( (val_X.shape[0],10))
results2 = np.zeros( (val_X.shape[0],10))
results3 = np.zeros( (val_X.shape[0],10))

results1 = results1 + model.predict(val_X/255.)
results2= results2 + model2.predict(val_X/255.)
results3 = results3 + model3.predict(val_X/255.)

val_model1 = np.argmax(results1,axis = 1)
val_model2 = np.argmax(results2,axis = 1)
val_model3 = np.argmax(results3,axis = 1)

val_model1 = val_model1.reshape(val_model1.shape[0],1)
val_model2 = val_model2.reshape(val_model2.shape[0],1)
val_model3 = val_model3.reshape(val_model3.shape[0],1)

val_DNN = np.concatenate((val_model1, val_model2), axis=1)
val_DNN = np.concatenate((val_DNN, val_model3), axis=1)


In [40]:
import scipy
labels_v = scipy.stats.mode(val_DNN, axis=1,nan_policy='propagate')[0]

labels_v = np.squeeze(labels_v)
labels_v = labels_v.reshape(val_Y.shape)

from sklearn.metrics import accuracy_score

accuracy = accuracy_score(labels_v, val_Y)
print(accuracy)

0.9992881548974943


In [ ]:
import scipy
test_csv = pd.read_csv("../input/Kannada-MNIST/test.csv")
data_test = np.array(test_csv.drop("id",axis=1),dtype=np.float32)
data_test = np.reshape(data_test,(-1,28,28,1))
X_test = data_test / 255.0
#X_test = data_test
print(X_test.shape,X_test.dtype)

In [ ]:
test_DNN = np.zeros( (X_test.shape[0],30))
test1 = np.zeros( (X_test.shape[0],10))
test2 = np.zeros( (X_test.shape[0],10))
test3 = np.zeros( (X_test.shape[0],10))

test1 = test1 + model.predict(X_test)
test2 = test2 + model2.predict(X_test)
test3 = test3 + model3.predict(X_test)

test1_ = np.argmax(test1,axis = 1)
test2_ = np.argmax(test2,axis = 1)
test3_ = np.argmax(test3,axis = 1)

test1_ = test1_.reshape(test1_.shape[0],1)
test2_ = test2_.reshape(test2_.shape[0],1)
test3_ = test3_.reshape(test3_.shape[0],1)

test_DNN = np.concatenate((test1_, test2_), axis=1)
test_DNN = np.concatenate((test_DNN, test3_), axis=1)

labels_test = scipy.stats.mode(test_DNN, axis=1,nan_policy='propagate')[0]

labels_test = np.squeeze(labels_test)
labels_test = labels_test.reshape((X_test.shape[0],1))

In [ ]:
submission = pd.read_csv("../input/Kannada-MNIST/sample_submission.csv")
submission['label'] = labels_test
submission.to_csv("submission.csv",index=False)

In [ ]:
submission.head()